In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

import sys
sys.path.insert(1, '/Users/bolin/Desktop/NUS_Y4S2/BT4222/BT4222_repo')
from custom_utils import *

In [2]:
df = pd.read_csv("/Users/bolin/Desktop/NUS_Y4S2/BT4222/BT4222_repo/data/cooked_data/cooked_complete_dataset.csv")
# prevent 0 division zero
df = df.replace(0, 1)

In [3]:
df

,date,Adj_Close_BTC-USD,Open_BTC-USD,High_BTC-USD,Low_BTC-USD,Volume_BTC-USD,Adj_Close_SPY,Adj_Close_GLD,Adj_Close_CHFUSD=X,Adj_Close_CNYUSD=X,Adj_Close_EURUSD=X,Adj_Close_GBPUSD=X,Adj_Close_JPYUSD=X,coindesk_sentiment,num_of_coindesk_posts,reddit_comments_sentiments,top_50_reddit_posts_sentiments,blockchain_transactions_per_block,blockchain_hash_rates
0,2020-12-14,19246.64453,19144.49219,19305.09961,19012.70898,2.247400e+10,361.926788,171.539993,1.125442,0.152772,1.213340,1.331824,0.009621,0.249489,12,0.188275,0.297238,2167.931034,134533587.6
1,2020-12-15,19417.07617,19246.91992,19525.00781,19079.84180,2.674198e+10,366.819824,173.940002,1.127930,0.152679,1.214890,1.333084,0.009614,0.173773,18,0.144389,0.399427,2288.857143,133351912.2
2,2020-12-16,21310.59766,19418.81836,21458.90820,19298.31641,4.440901e+10,367.395508,174.899994,1.129382,0.152945,1.215430,1.344447,0.009649,0.341491,11,0.137256,0.489673,2204.314685,132323572.3
3,2020-12-17,22805.16211,21308.35156,23642.66016,21234.67578,7.137861e+10,369.449982,176.740005,1.129446,0.153109,1.219959,1.350293,0.009664,0.197572,10,0.156723,0.636030,2399.077519,132373208.7
4,2020-12-18,23137.96094,22806.79688,23238.60156,22399.81250,4.038790e+10,367.974792,176.440002,1.130301,0.153090,1.226272,1.357018,0.009696,0.315601,2,0.166419,0.107093,2392.031847,131791042.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,2021-04-01,58918.83203,58930.27734,59930.02734,57726.41797,6.552083e+10,400.609985,161.979996,1.059434,0.152644,1.172883,1.378626,0.009029,0.277144,17,0.094478,0.202896,NaN,NaN
109,2021-04-02,59384.31250,59098.87891,60267.18750,58869.28125,5.872786e+10,400.609985,161.979996,1.061447,0.152337,1.177995,1.383509,0.009041,0.121955,10,0.146423,0.152625,NaN,NaN
110,2021-04-03,59384.31250,59098.87891,60267.18750,58869.28125,5.872786e+10,400.609985,161.979996,1.061346,0.152337,1.176886,1.382915,0.009038,0.501192,1,0.132042,0.274113,NaN,NaN
111,2021-04-04,58758.55469,57604.83984,58913.74609,57168.67578,5.074966e+10,400.609985,161.979996,1.061346,0.152337,1.176886,1.382915,0.009038,0.206128,1,0.216657,0.148517,NaN,NaN


In [4]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [5]:
df.columns

Index(['date', 'Adj_Close_BTC-USD', 'Open_BTC-USD', 'High_BTC-USD',
       'Low_BTC-USD', 'Volume_BTC-USD', 'Adj_Close_SPY', 'Adj_Close_GLD',
       'Adj_Close_CHFUSD=X', 'Adj_Close_CNYUSD=X', 'Adj_Close_EURUSD=X',
       'Adj_Close_GBPUSD=X', 'Adj_Close_JPYUSD=X', 'coindesk_sentiment',
       'num_of_coindesk_posts', 'reddit_comments_sentiments',
       'top_50_reddit_posts_sentiments', 'blockchain_transactions_per_block',
       'blockchain_hash_rates'],
      dtype='object')

In [6]:
# conduct log return transformation on all columns
df = log_return_transformation(df)

In [7]:
# custom number of lags for selected columns
# columns that are not specify in this dictionary will be lagged by on by default
lags = {
    'Adj_Close_BTC-USD': 6,
    'Open_BTC-USD': 6,
    'Low_BTC-USD': 6,
    'Volume_BTC-USD': 6,
    'Adj_Close_SPY': 6,
    'Adj_Close_GLD': 6,
}

In [8]:
df = get_num_lags(df, lags)

/Users/bolin/Desktop/NUS_Y4S2/BT4222/BT4222_repo/custom_utils/get_num_lags.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column + '_lag_' + str(i)] = df[column].shift(i)


In [9]:
df.columns

Index(['date', 'Adj_Close_BTC-USD', 'Open_BTC-USD_lag_1', 'Open_BTC-USD_lag_2',
       'Open_BTC-USD_lag_3', 'Open_BTC-USD_lag_4', 'Open_BTC-USD_lag_5',
       'High_BTC-USD_lag_1', 'Low_BTC-USD_lag_1', 'Low_BTC-USD_lag_2',
       'Low_BTC-USD_lag_3', 'Low_BTC-USD_lag_4', 'Low_BTC-USD_lag_5',
       'Volume_BTC-USD_lag_1', 'Volume_BTC-USD_lag_2', 'Volume_BTC-USD_lag_3',
       'Volume_BTC-USD_lag_4', 'Volume_BTC-USD_lag_5', 'Adj_Close_SPY_lag_1',
       'Adj_Close_SPY_lag_2', 'Adj_Close_SPY_lag_3', 'Adj_Close_SPY_lag_4',
       'Adj_Close_SPY_lag_5', 'Adj_Close_GLD_lag_1', 'Adj_Close_GLD_lag_2',
       'Adj_Close_GLD_lag_3', 'Adj_Close_GLD_lag_4', 'Adj_Close_GLD_lag_5',
       'Adj_Close_CHFUSD=X_lag_1', 'Adj_Close_CNYUSD=X_lag_1',
       'Adj_Close_EURUSD=X_lag_1', 'Adj_Close_GBPUSD=X_lag_1',
       'Adj_Close_JPYUSD=X_lag_1', 'coindesk_sentiment_lag_1',
       'num_of_coindesk_posts_lag_1', 'reddit_comments_sentiments_lag_1',
       'top_50_reddit_posts_sentiments_lag_1',
       'blo

In [10]:
df['Adj_Close_BTC-USD'] = df['Adj_Close_BTC-USD'].apply(lambda x: 1 if x > 0 else 0)

In [12]:
train = df[df['date']<'2021-04-01'].sort_values('date')
test = df[df['date']>'2021-03-31'].sort_values('date')

In [14]:
train

,date,Adj_Close_BTC-USD,Open_BTC-USD_lag_1,Open_BTC-USD_lag_2,Open_BTC-USD_lag_3,Open_BTC-USD_lag_4,Open_BTC-USD_lag_5,High_BTC-USD_lag_1,Low_BTC-USD_lag_1,Low_BTC-USD_lag_2,...,Adj_Close_CNYUSD=X_lag_1,Adj_Close_EURUSD=X_lag_1,Adj_Close_GBPUSD=X_lag_1,Adj_Close_JPYUSD=X_lag_1,coindesk_sentiment_lag_1,num_of_coindesk_posts_lag_1,reddit_comments_sentiments_lag_1,top_50_reddit_posts_sentiments_lag_1,blockchain_transactions_per_block_lag_1,blockchain_hash_rates_lag_1
6,2020-12-20,0,0.014196,0.067960,0.092856,0.008892,0.005336,0.035810,0.018868,0.053417,...,0.000000,0.000000,0.000000,0.000000,-0.211596,1.252763,-0.566069,1.359533,-0.095446,0.015375
7,2020-12-21,0,0.031023,0.014196,0.067960,0.092856,0.008892,0.005127,0.013975,0.018868,...,0.000000,0.000000,0.000000,0.000000,0.068756,0.251314,0.444461,0.195889,-0.151917,0.022101
8,2020-12-22,1,-0.016365,0.031023,0.014196,0.067960,0.092856,-0.006202,-0.043636,0.013975,...,-0.000597,0.000000,-0.012474,-0.003156,-0.063549,0.000000,0.192906,0.061448,0.060372,-0.008725
9,2020-12-23,0,-0.029414,-0.016365,0.031023,0.014196,0.067960,-0.011289,0.012166,-0.043636,...,-0.001865,-0.002107,0.003544,0.001132,0.202284,0.441833,-0.478721,-0.012084,0.133250,-0.008837
10,2020-12-24,1,0.042429,-0.029414,-0.016365,0.031023,0.014196,0.009813,0.016450,0.012166,...,0.000917,-0.004115,-0.003732,-0.001875,-0.199405,-0.336472,0.339762,-0.408491,-0.030940,0.002954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,2021-03-27,1,-0.019994,-0.036933,0.003641,-0.053681,-0.013676,0.032158,0.014122,-0.032077,...,-0.004625,-0.003090,0.004390,-0.003569,-0.266282,-0.133531,0.630828,0.233651,0.042733,0.004575
104,2021-03-28,0,0.064702,-0.019994,-0.036933,0.003641,-0.053681,0.025621,0.050341,0.014122,...,0.000000,0.000000,0.000000,0.000000,1.663437,-2.639057,-1.026719,-0.810804,-0.174642,-0.009067
105,2021-03-29,1,0.015073,0.064702,-0.019994,-0.036933,0.003641,0.000744,0.015153,0.050341,...,0.000000,0.000000,0.000000,0.000000,-2.858168,0.000000,0.589084,-1.657930,-0.147994,0.004688
106,2021-03-30,1,-0.000483,0.015073,0.064702,-0.019994,-0.036933,0.030133,0.001238,0.015153,...,0.000657,0.000966,0.003043,-0.005590,1.345260,2.833213,-0.117053,1.592596,0.250498,0.002802


In [15]:
y_train = train['Adj_Close_BTC-USD']
X_train = train.iloc[:,2:]
X_train_no_sentiment = X_train.drop(['coindesk_sentiment_lag_1', 'num_of_coindesk_posts_lag_1', 'reddit_comments_sentiments_lag_1', 'top_50_reddit_posts_sentiments_lag_1'], axis=1)

In [17]:
y_train.value_counts()

1    57
0    45
Name: Adj_Close_BTC-USD, dtype: int64

### Logistic regression with complete set of independent variables including sentiments

In [26]:
lr = LogisticRegression(max_iter=500, solver='liblinear')
lr.fit(X_train, y_train)

LogisticRegression(max_iter=500, solver='liblinear')

In [27]:
y_pred_class = lr.predict(X_train)
print('Accuracy score in training set: {}'.format(metrics.accuracy_score(y_train, y_pred_class)))
print('Precision score in training set: {}'.format(metrics.precision_score(y_train, y_pred_class)))
print('Recall score in training set: {}'.format(metrics.recall_score(y_train, y_pred_class)))
print('Confusion matrix:')
print(metrics.confusion_matrix(y_train, y_pred_class))

y_pred_prob = lr.predict_proba(X_train)[:, 1]
print('ROC_AUC score in training set: {}'.format(metrics.roc_auc_score(y_train, y_pred_prob)))

print('*'*80)

# y_pred_class = lr.predict(X_test)
# print('Accuracy score in test set: {}'.format(metrics.accuracy_score(y_test, y_pred_class)))
# print('Precision score in test set: {}'.format(metrics.precision_score(y_test, y_pred_class)))
# print('Recall score in test set: {}'.format(metrics.recall_score(y_test, y_pred_class)))
# print('Confusion matrix:')
# print(metrics.confusion_matrix(y_test, y_pred_class))

# y_pred_prob = lr.predict_proba(X_test)[:, 1]
# print('ROC_AUC score in test set: {}'.format(metrics.roc_auc_score(y_test, y_pred_prob)))

# print("userful_funny_cool is useful.")

Accuracy score in training set: 0.6470588235294118
Precision score in training set: 0.6521739130434783
Recall score in training set: 0.7894736842105263
Confusion matrix:
[[21 24]
 [12 45]]
ROC_AUC score in training set: 0.6732943469785575
********************************************************************************


### Logistic regression with complete set of independent variables excluding sentiments

In [28]:
lr = LogisticRegression(max_iter=500, solver='liblinear')
lr.fit(X_train_no_sentiment, y_train)

LogisticRegression(max_iter=500, solver='liblinear')

In [29]:
y_pred_class = lr.predict(X_train_no_sentiment)
print('Accuracy score in training set: {}'.format(metrics.accuracy_score(y_train, y_pred_class)))
print('Precision score in training set: {}'.format(metrics.precision_score(y_train, y_pred_class)))
print('Recall score in training set: {}'.format(metrics.recall_score(y_train, y_pred_class)))
print('Confusion matrix:')
print(metrics.confusion_matrix(y_train, y_pred_class))

y_pred_prob = lr.predict_proba(X_train_no_sentiment)[:, 1]
print('ROC_AUC score in training set: {}'.format(metrics.roc_auc_score(y_train, y_pred_prob)))

print('*'*80)

# y_pred_class = lr.predict(X_test)
# print('Accuracy score in test set: {}'.format(metrics.accuracy_score(y_test, y_pred_class)))
# print('Precision score in test set: {}'.format(metrics.precision_score(y_test, y_pred_class)))
# print('Recall score in test set: {}'.format(metrics.recall_score(y_test, y_pred_class)))
# print('Confusion matrix:')
# print(metrics.confusion_matrix(y_test, y_pred_class))

# y_pred_prob = lr.predict_proba(X_test)[:, 1]
# print('ROC_AUC score in test set: {}'.format(metrics.roc_auc_score(y_test, y_pred_prob)))

# print("userful_funny_cool is useful.")

Accuracy score in training set: 0.5784313725490197
Precision score in training set: 0.5795454545454546
Recall score in training set: 0.8947368421052632
Confusion matrix:
[[ 8 37]
 [ 6 51]]
ROC_AUC score in training set: 0.621832358674464
********************************************************************************


### Logistic regression with selected set of independent variables from stepwise regression

In [30]:
selected_x_variables = "coindesk_sentiment_lag_1 + Adj_Close_GLD_lag_2 + top_50_reddit_posts_sentiments_lag_1 + Volume_BTC-USD_lag_3 + blockchain_transactions_per_block_lag_1"
selected_x_variables = selected_x_variables.split(' + ')

In [31]:
selected_x_variables

['coindesk_sentiment_lag_1',
 'Adj_Close_GLD_lag_2',
 'top_50_reddit_posts_sentiments_lag_1',
 'Volume_BTC-USD_lag_3',
 'blockchain_transactions_per_block_lag_1']

In [32]:
new_X_train = X_train[selected_x_variables]

In [33]:
lr = LogisticRegression(max_iter=500, solver='liblinear')
lr.fit(new_X_train, y_train)

LogisticRegression(max_iter=500, solver='liblinear')

In [34]:
y_pred_class = lr.predict(new_X_train)
print('Accuracy score in training set: {}'.format(metrics.accuracy_score(y_train, y_pred_class)))
print('Precision score in training set: {}'.format(metrics.precision_score(y_train, y_pred_class)))
print('Recall score in training set: {}'.format(metrics.recall_score(y_train, y_pred_class)))
print('Confusion matrix:')
print(metrics.confusion_matrix(y_train, y_pred_class))

y_pred_prob = lr.predict_proba(new_X_train)[:, 1]
print('ROC_AUC score in training set: {}'.format(metrics.roc_auc_score(y_train, y_pred_prob)))

print('*'*80)

# y_pred_class = lr.predict(X_test)
# print('Accuracy score in test set: {}'.format(metrics.accuracy_score(y_test, y_pred_class)))
# print('Precision score in test set: {}'.format(metrics.precision_score(y_test, y_pred_class)))
# print('Recall score in test set: {}'.format(metrics.recall_score(y_test, y_pred_class)))
# print('Confusion matrix:')
# print(metrics.confusion_matrix(y_test, y_pred_class))

# y_pred_prob = lr.predict_proba(X_test)[:, 1]
# print('ROC_AUC score in test set: {}'.format(metrics.roc_auc_score(y_test, y_pred_prob)))

# print("userful_funny_cool is useful.")

Accuracy score in training set: 0.6274509803921569
Precision score in training set: 0.6266666666666667
Recall score in training set: 0.8245614035087719
Confusion matrix:
[[17 28]
 [10 47]]
ROC_AUC score in training set: 0.656140350877193
********************************************************************************
